# Introduction

### Problem

Huang, Zhe, Wessler, Benjamin S., and Hughes, Michael C. (2023) – Detecting Heart
Disease from Multi-View Ultrasound Images via Supervised Attention Multiple Instance
Learning describes the clinical problem of under-diagnosis and under-treatment
of aortic stenosis (AS), a degenerative valve condition. In clinical practice, AS is diagnosed
by manual expert view of a transthoracic echocardiography (TTE) – which uses ultrasound
to produce many images of the heart. AS can be treated effectively, but requires
identification early on. Automatic screening of AS from transthoracic echocardiography
imagery can improve the rate of detection and decrease mortality.

### Paper Explanation
The challenge with automatic detection is each TTE “consists of dozens of images or videos
(typically 27-97 in our data) that show
the heart’s complex anatomy from different acquisition angles” (Huang, Wessler, and
Hughes, 2023) where a clinical expert identifies imagery where the aortic valve is clearly
visible, then assesses the severity on a 3-level scale (no, early, significant disease).
Traditional Deep Learning approaches classify a single image with a single result, however
the clinical expert review makes a single “coherent prediction” (Huang, Wessler, and
Hughes, 2023) from knowledge gathered from the set of images. Additionally, the image
views produced by a TTE are often unlabeled in Electronic Health Records, further
complicating any Deep Learning approaches.

The paper finds previous approaches to automatic detection such as attention-based
multiple instance learning (MIL) to be insufficient based on accuracy and detection yield,
and explores a novel MIL approach to improve the detection of AS from automatic detection
that mimics the methodology of a clinical expert.

The paper outlines two novel contributions to automatic AS detection:

1) Supervised attention mechanism that identifies relevant TTE views (often unlabeled),
mimicking human filtering done by a clinical expert. This is accomplished by introducing a
new loss term, “supervised attention (SA)”, to match attention weights to the relevance
scores from a View Relevance classifier.

2) Self-supervised pretraining strategy through contrastive learning on the embedding of
the entire TTE study (i.e., a “bag of images”) – compared to traditional pretraining
strategies which focus on individual images.

### Paper Results

The paper uses *balanced accuracy* as the performance metric due to the class imbalance in the TMED-2 dataset -- making standard accuracy "less suitable" (Huang et al., 2023). The proposed method (SAMIL) was compared to general-purpose multi-instance algorithms and prior methods for AS diagnosis using deep neural networks.

SAMIL performed much better (76% balanced accuracy) than 4 other state-of-the-art attention-based MIL architectures tested vs. a range of 60-67% balanced accuracy for existing algorithms.

The chart below from the original paper outlines the balanced accuracy of SAMIL against other  approaches dedicated to AS diagnosis (*Filter then Average* and *Weighted Average by View Relevance*), and other general approaches including ABMIL, Set Transformer, and DSMIL.

![](https://drive.google.com/uc?export=view&id=1IpY4tUQAVBqCfwaPM73vmP8bVcFHnqe6)

(Huang et al., 2023)

# Scope of Reproducibility:

### Hypothese To Be Tested

1. A supervised attention mechanism will provide significant improvements over
standard MIL approaches in AS detection rates and detection accuracy, with a
smaller model size.
2. Self-supervised pretraining of “study-level” TTE artifacts provides improvements in
AS detection rates and detection accuracy over traditional “image-level” pretraining,
or no pretraining at all.

### Planned Ablations

The paper has two ablations targeting the attention strategy and the pretraining strategy.

1. Attention: The attention mechanisms within the pooling layer σ to be tested are the
baseline ABMIL model, ABMIL with gated attention, and the SAMIL model without
pretraining. The paper compares the performance of these three approaches and identifies
that SAMIL’s supervised attention model outperforms ABMIL (the baseline model that
SAMIL builds upon) by +1200 bps. The Github repo scripts includes parameters to control
the attention mechanism for ABMIL (gated_attention vs. attention), and SAMIL with and
without pretraining.
2. Pre Training: The paper introduces a novel approach of built-in study-level (i.e., bag-level)
pretraining. This ablation compares different pretraining strategies including: image-level
contrastive learning and no pretraining to the study-level pretraining approach. The paper
finds no improvements with image-level pretraining, but the study-level pretraining shows
improvements of +480 bps. The Github repo scripts include parameters to control
pretraining options of: study level, image level, and none.

# Methodology

To reproduce this paper, the following pre-requisites must be acquired:

1. Access to the TMED-2 dataset [here](https://tmed.cs.tufts.edu/tmed_v2.html)
2. Download the pretrained view classifiers, MOCO pretrained checkpoints, and training curves of SAMIL from the paper's Github repo [here](https://tufts.box.com/s/c5w8123j7h3dpls75jye1363uh8qv8us). Once downloaded, upload the entire unzipped folder to your Google Drive (see path below).

In [ ]:
import os
import zipfile
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Modify the paths below if they differ from your upload locations.
# Assumes the SAMIL Github repo has been cloned and uploaded to drive in the `SAMIL` folder.
MODEL_CHECKPOINTS = '/content/drive/MyDrive/SAMIL/model_checkpoints'
ROOT_DIR = '/content/drive/MyDrive/SAMIL'
DATA_INFO_DIR = '/content/drive/MyDrive/SAMIL/data_info'
DATA_DIR = '/content/drive/MyDrive/DL4H-TMED2/'

with zipfile.ZipFile(DATA_DIR + 'labeled.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

with zipfile.ZipFile(DATA_DIR + 'unlabeled.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

LOCAL_DATA_DIR = '/content/data/'

## Environment Setup

In [6]:
import math
import glob
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from tqdm import tqdm

trainingSeed = 0
batchSize = 1
numWorkers = 8
random.seed(trainingSeed)
np.random.seed(trainingSeed)
torch.manual_seed(trainingSeed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##  Data
The dataset used by the paper is the[ TMED-2 dataset](https://tmed.cs.tufts.edu/tmed_v2.html), containing transthoracic echocardiogram (TTE) imagery from routine care of patients at Tufts Medical Center.

The paper uses the (`view_and_diagnosis_labeled_set`) from TMED-2, consisting of 599 studies from 577 patients. The patients are labeled by board certified medical staff with the following values: none, early AS, or significant AS. The dataset has been partioned into different splits, each containing 360 training studies, 119 validation studies, and 120 test studies.

Code blocks below should only be executed after you have acquired the TMED-2 dataset and uploaded the `view_and_diagnosis_labeled_set` folder to Drive.

In [7]:
labeled_dir = '/content/drive/MyDrive/DL4H-TMED2/labeled'
unlabeled_dir = '/content/drive/MyDrive/DL4H-TMED2/unlabeled'

# Assumes the `view_and_diagnosis_labeled_set` is uploaded to a folder named `DL4H-TMED2`

TMED2SummaryTable = pd.read_csv(os.path.join(DATA_INFO_DIR, 'TMED2SummaryTable.csv'))
SEED_DIR = DATA_INFO_DIR + '/DataPartition/seed0/DEV479/FullyLabeledSet_studies'

train_PatientStudy_list = pd.read_csv(os.path.join(SEED_DIR, "train_studies.csv"))
val_PatientStudy_list = pd.read_csv(os.path.join(SEED_DIR, "val_studies.csv"))
test_PatientStudy_list = pd.read_csv(os.path.join(SEED_DIR, "test_studies.csv"))

train_PatientStudy_ids = train_PatientStudy_list["study"].values
val_PatientStudy_ids = val_PatientStudy_list["study"].values
test_PatientStudy_ids = test_PatientStudy_list["study"].values

#Debug
#print(train_PatientStudy_ids)
# print(val_PatientStudy_ids)
# print(test_PatientStudy_ids)

## EchoDataset

The class below is directly from the paper repo and handles transforming and loading the TMED-2 image data.

In [8]:
from PIL import Image
from torch.utils.data import Dataset

DiagnosisStr_to_Int_Mapping={
    'no_AS':0,
    'mild_AS':1,
    'mildtomod_AS':1,
    'moderate_AS':2,
    'severe_AS':2

}

class EchoDataset(Dataset):
    def __init__(self, PatientStudy_list, TMED2SummaryTable, ML_DATA_dir, sampling_strategy='first_frame', training_seed=0, transform_fn=None):

        self.PatientStudy_list = PatientStudy_list
        self.TMED2SummaryTable = TMED2SummaryTable #note: using the patient_id column in TMED2SummaryTable can uniquely identify a patient_study (there is NO same patient_study belong to different parts: diagnosis_labeled/, unlabeled/, view_and_diagnosis_labeled_set/, view_labeled AT THE SAME TIME)

        self.ML_DATA_dir = ML_DATA_dir

        self.sampling_strategy = sampling_strategy

        self.training_seed=training_seed

        self.transform_fn = transform_fn

        self.bag_of_PiatentStudy_images, self.bag_of_PatientStudy_DiagnosisLabels = self._create_bags()



    def _create_bags(self):

        bag_of_PatientStudy_images = []
        bag_of_PatientStudy_DiagnosisLabels = []

        for PatientStudy in self.PatientStudy_list:
            this_PatientStudyRecords_from_TMED2SummaryTable = self.TMED2SummaryTable[self.TMED2SummaryTable['patient_study']==PatientStudy]
            assert this_PatientStudyRecords_from_TMED2SummaryTable.shape[0]!=0, 'every PatientStudy from the studylist should be found in TMED2SummaryTable'

            this_PatientStudyRecords_from_TMED2SummaryTable_DiagnosisLabel = list(set(this_PatientStudyRecords_from_TMED2SummaryTable.diagnosis_label.values))
            assert len(this_PatientStudyRecords_from_TMED2SummaryTable_DiagnosisLabel)==1, 'every PatientStudy should only have one diagnosis label'

            this_PatientStudy_DiagnosisLabel = this_PatientStudyRecords_from_TMED2SummaryTable_DiagnosisLabel[0]
            this_PatientStudy_DiagnosisLabel = DiagnosisStr_to_Int_Mapping[this_PatientStudy_DiagnosisLabel]

            this_PatientStudy_Id_ImagesPattern = PatientStudy + "_*.png"
            this_PatientStudy_Id_LabeledImages: list[str] = glob.glob(pathname=this_PatientStudy_Id_ImagesPattern, root_dir=os.path.join(self.ML_DATA_dir, "labeled"))
            this_PatientStudy_Id_UnlabeledImages: list[str] = glob.glob(pathname=this_PatientStudy_Id_ImagesPattern, root_dir=os.path.join(self.ML_DATA_dir, "unlabeled"))

            # From paper repo, sort to ensure order of images are consistent each run.
            this_PatientStudy_Id_LabeledImages.sort()
            this_PatientStudy_Id_UnlabeledImages.sort()

            this_PatientStudyImages = []

            for ImagePath in this_PatientStudy_Id_LabeledImages:
                this_PatientStudyImages.append(
                    np.array(Image.open(self.ML_DATA_dir + '/labeled/' + ImagePath).convert(mode="RGB"))
                )

            for ImagePath in this_PatientStudy_Id_UnlabeledImages:
                this_PatientStudyImages.append(
                    np.array(Image.open(self.ML_DATA_dir + '/unlabeled/' + ImagePath).convert(mode="RGB"))
                )

            bag_of_PatientStudy_images.append(np.array(this_PatientStudyImages))
            bag_of_PatientStudy_DiagnosisLabels.append(this_PatientStudy_DiagnosisLabel)

        return bag_of_PatientStudy_images, bag_of_PatientStudy_DiagnosisLabels


    def __len__(self):
        return len(self.bag_of_PiatentStudy_images)


    def __getitem__(self, index):

        bag_image = self.bag_of_PiatentStudy_images[index]

        if self.transform_fn is not None:
            bag_image = torch.stack([self.transform_fn(Image.fromarray(image)) for image in bag_image])


        DiagnosisLabel = self.bag_of_PatientStudy_DiagnosisLabels[index]

        return bag_image, DiagnosisLabel



### Transformations

In [9]:
import PIL
import PIL.ImageOps
import PIL.ImageEnhance
import PIL.ImageDraw
from PIL import Image

PARAMETER_MAX = 10


def AutoContrast(img, **kwarg):
    return PIL.ImageOps.autocontrast(img)


def Brightness(img, v, max_v, bias=0):
    v = _float_parameter(v, max_v) + bias
    return PIL.ImageEnhance.Brightness(img).enhance(v)


def Color(img, v, max_v, bias=0):
    v = _float_parameter(v, max_v) + bias
    return PIL.ImageEnhance.Color(img).enhance(v)


def Contrast(img, v, max_v, bias=0):
    v = _float_parameter(v, max_v) + bias
    return PIL.ImageEnhance.Contrast(img).enhance(v)


def Cutout(img, v, max_v, bias=0):
    if v == 0:
        return img
    v = _float_parameter(v, max_v) + bias
    v = int(v * min(img.size))
    return CutoutAbs(img, v)


def CutoutAbs(img, v, **kwarg):
    w, h = img.size
    x0 = np.random.uniform(0, w)
    y0 = np.random.uniform(0, h)
    x0 = int(max(0, x0 - v / 2.))
    y0 = int(max(0, y0 - v / 2.))
    x1 = int(min(w, x0 + v))
    y1 = int(min(h, y0 + v))
    xy = (x0, y0, x1, y1)
    # gray
    color = (127, 127, 127)
    img = img.copy()
    PIL.ImageDraw.Draw(img).rectangle(xy, color)
    return img


def Equalize(img, **kwarg):
    return PIL.ImageOps.equalize(img)


def Identity(img, **kwarg):
    return img


def Invert(img, **kwarg):
    return PIL.ImageOps.invert(img)


def Posterize(img, v, max_v, bias=0):
    v = _int_parameter(v, max_v) + bias
    return PIL.ImageOps.posterize(img, v)


def Rotate(img, v, max_v, bias=0):
    v = _int_parameter(v, max_v) + bias
    if random.random() < 0.5:
        v = -v
    return img.rotate(v)


def Sharpness(img, v, max_v, bias=0):
    v = _float_parameter(v, max_v) + bias
    return PIL.ImageEnhance.Sharpness(img).enhance(v)


def ShearX(img, v, max_v, bias=0):
    v = _float_parameter(v, max_v) + bias
    if random.random() < 0.5:
        v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, v, 0, 0, 1, 0))


def ShearY(img, v, max_v, bias=0):
    v = _float_parameter(v, max_v) + bias
    if random.random() < 0.5:
        v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, v, 1, 0))


def Solarize(img, v, max_v, bias=0):
    v = _int_parameter(v, max_v) + bias
    return PIL.ImageOps.solarize(img, 256 - v)


def SolarizeAdd(img, v, max_v, bias=0, threshold=128):
    v = _int_parameter(v, max_v) + bias
    if random.random() < 0.5:
        v = -v
    img_np = np.array(img).astype(np.int)
    img_np = img_np + v
    img_np = np.clip(img_np, 0, 255)
    img_np = img_np.astype(np.uint8)
    img = Image.fromarray(img_np)
    return PIL.ImageOps.solarize(img, threshold)


def TranslateX(img, v, max_v, bias=0):
    v = _float_parameter(v, max_v) + bias
    if random.random() < 0.5:
        v = -v
    v = int(v * img.size[0])
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, v, 0, 1, 0))


def TranslateY(img, v, max_v, bias=0):
    v = _float_parameter(v, max_v) + bias
    if random.random() < 0.5:
        v = -v
    v = int(v * img.size[1])
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, 0, 1, v))


def _float_parameter(v, max_v):
    return float(v) * max_v / PARAMETER_MAX


def _int_parameter(v, max_v):
    return int(v * max_v / PARAMETER_MAX)


def fixmatch_augment_pool():
    # FixMatch paper
    augs = [(AutoContrast, None, None),
            (Brightness, 0.9, 0.05),
            (Color, 0.9, 0.05),
            (Contrast, 0.9, 0.05),
            (Equalize, None, None),
            (Identity, None, None),
            (Posterize, 4, 4),
            (Rotate, 30, 0),
            (Sharpness, 0.9, 0.05),
            (ShearX, 0.3, 0),
            (ShearY, 0.3, 0),
            (Solarize, 256, 0),
            (TranslateX, 0.3, 0),
            (TranslateY, 0.3, 0)]
    return augs

class RandAugmentMC(object):
    def __init__(self, n, m):
        assert n >= 1
        assert 1 <= m <= 10
        self.n = n
        self.m = m
        self.augment_pool = fixmatch_augment_pool()

    def __call__(self, img):
        ops = random.choices(self.augment_pool, k=self.n)
        for op, max_v, bias in ops:
            v = np.random.randint(1, self.m)
            if random.random() < 0.5:
                img = op(img, v=v, max_v=max_v, bias=bias)
        img = CutoutAbs(img, int(32*0.5))
        return img


In [10]:
transform_eval = transforms.Compose([
    transforms.ToTensor(),
])

transform_labeledtrain = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=112,
                          padding=int(112*0.125),
                          padding_mode='reflect'),
    RandAugmentMC(n=2, m=10),
    transforms.ToTensor(),
])

## Create Dataset

In [11]:
train_dataset = EchoDataset(train_PatientStudy_ids, TMED2SummaryTable, LOCAL_DATA_DIR, sampling_strategy='first_frame', training_seed=trainingSeed, transform_fn=transform_labeledtrain)

trainmemory_dataset = EchoDataset(train_PatientStudy_ids, TMED2SummaryTable, LOCAL_DATA_DIR, sampling_strategy='first_frame', training_seed=trainingSeed, transform_fn=transform_eval)

val_dataset = EchoDataset(val_PatientStudy_ids, TMED2SummaryTable, LOCAL_DATA_DIR, sampling_strategy='first_frame', training_seed=trainingSeed, transform_fn=transform_eval)

test_dataset = EchoDataset(test_PatientStudy_ids, TMED2SummaryTable, LOCAL_DATA_DIR, sampling_strategy='first_frame', training_seed=trainingSeed, transform_fn=transform_eval)

In [12]:
print("train: {}, trainmemory: {}, val: {}, test: {}".format(len(train_dataset), len(trainmemory_dataset), len(val_dataset), len(test_dataset)))


train: 360, trainmemory: 360, val: 119, test: 120


In [13]:
train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True, num_workers=numWorkers)
trainmemory_loader = DataLoader(trainmemory_dataset, batch_size=batchSize, shuffle=False, num_workers=numWorkers)
val_loader = DataLoader(val_dataset, batch_size=batchSize, shuffle=False, num_workers=numWorkers)
test_loader = DataLoader(test_dataset, batch_size=batchSize, shuffle=False, num_workers=numWorkers)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## Dataset Analysis

The method below provides a high level analysis of the TMED-2 dataset for the seed0 split.

We see number of samples and classes are as expected across the Train, Validation, and Test sets based on the specs from the TMED-2 website.

Additionally, we see an imbalancedclass distribution. Namely, `sev_AS` which maps to a diagnosis of "Severe AS" dominates the dataset, while `no_AS` is poorly represented and could lead towards a biased prediction of `sev_AS`.

In [14]:
def dataset_statistics(loader, name):
    num_batches = len(loader)
    num_samples = len(loader.dataset)
    label_mapping = {0: "no_AS", 1: "mod_AS", 2: "sev_AS"}

    print(f"{name} DataLoader:")
    print(f"  Total number of batches: {num_batches}")
    print(f"  Total number of samples: {num_samples}")

    labels = []
    for _, batch_labels in loader:
        labels.extend(batch_labels.tolist())
    num_classes = len(set(labels))
    print(f"  Number of classes: {num_classes}")

    class_distribution = {label_mapping[label]: labels.count(label) for label in set(labels) if label in label_mapping}
    print(f"  Class distribution: {class_distribution}")

dataset_statistics(train_loader, "Train")
dataset_statistics(trainmemory_loader, "Train Memory")
dataset_statistics(val_loader, "Validation")
dataset_statistics(test_loader, "Test")

Train DataLoader:
  Total number of batches: 360
  Total number of samples: 360


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  Number of classes: 3
  Class distribution: {'no_AS': 76, 'mod_AS': 103, 'sev_AS': 181}
Train Memory DataLoader:
  Total number of batches: 360
  Total number of samples: 360
  Number of classes: 3
  Class distribution: {'no_AS': 76, 'mod_AS': 103, 'sev_AS': 181}
Validation DataLoader:
  Total number of batches: 119
  Total number of samples: 119
  Number of classes: 3
  Class distribution: {'no_AS': 25, 'mod_AS': 34, 'sev_AS': 60}
Test DataLoader:
  Total number of batches: 120
  Total number of samples: 120
  Number of classes: 3
  Class distribution: {'no_AS': 26, 'mod_AS': 34, 'sev_AS': 60}


##   Model

The paper evaluates multiple models in addition to the SAMIL model they are proposing. These include ABMIL, Set Transformer, and DSMIL.

### SAMIL

#### View Classifier

The following class for the View Classifier is directly ported from the paper's Github repo.

In [15]:
import logging

import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)



def mish(x):
    """Mish: A Self Regularized Non-Monotonic Neural Activation Function (https://arxiv.org/abs/1908.08681)"""
    return x * torch.tanh(F.softplus(x))


class PSBatchNorm2d(nn.BatchNorm2d):
    """How Does BN Increase Collapsed Neural Network Filters? (https://arxiv.org/abs/2001.11216)"""

    def __init__(self, num_features, alpha=0.1, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True):
        super().__init__(num_features, eps, momentum, affine, track_running_stats)
        self.alpha = alpha

    def forward(self, x):
        return super().forward(x) + self.alpha


class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, drop_rate=0.0, activate_before_residual=False):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes, momentum=0.001)
        self.relu1 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes, momentum=0.001)
        self.relu2 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.drop_rate = drop_rate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                                                                padding=0, bias=False) or None
        self.activate_before_residual = activate_before_residual

    def forward(self, x):
        if not self.equalInOut and self.activate_before_residual == True:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.drop_rate > 0:
            out = F.dropout(out, p=self.drop_rate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)


class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, drop_rate=0.0, activate_before_residual=False):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(
            block, in_planes, out_planes, nb_layers, stride, drop_rate, activate_before_residual)

    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, drop_rate, activate_before_residual):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes,
                                i == 0 and stride or 1, drop_rate, activate_before_residual))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)

# args.model_depth = 28
# args.model_width = 2

class WideResNet(nn.Module):
    def __init__(self, num_classes, depth=28, widen_factor=2, drop_rate=0.0):
        super(WideResNet, self).__init__()
        channels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor, 128*widen_factor]
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) / 6 #equivalent to 'repeat' in tf repo
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, channels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(
            n, channels[0], channels[1], block, 1, drop_rate, activate_before_residual=True)
        # 2nd block
        self.block2 = NetworkBlock(
            n, channels[1], channels[2], block, 2, drop_rate)
        # 3rd block
        self.block3 = NetworkBlock(
            n, channels[2], channels[3], block, 2, drop_rate)

        # 4th block (hz added)
        self.block4 = NetworkBlock(
            n, channels[3], channels[4], block, 2, drop_rate)

        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(channels[4], momentum=0.001)
        self.relu = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.fc = nn.Linear(channels[4], num_classes)
        self.channels = channels[4]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='leaky_relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1.0)
                nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0.0)

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)

        out = self.relu(self.bn1(out))
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(-1, self.channels)
        return self.fc(out)


def build_wideresnet(depth, widen_factor, dropout, num_classes):
    logger.info(f"Model: WideResNet {depth}x{widen_factor}")
    return WideResNet(depth=depth,
                      widen_factor=widen_factor,
                      drop_rate=dropout,
                      num_classes=num_classes)


#### SAMIL Model

The following class represents the SAMIL model. This code is unchanged from the paper's Github repo.

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SAMIL(nn.Module):
    def __init__(self, num_classes=3):
        super(SAMIL, self).__init__()
        self.L = 500
        self.B = 250
        self.D = 128
        self.K = 1
        self.num_classes = num_classes

        self.feature_extractor_part1 = nn.Sequential(
#             nn.Conv2d(1, 20, kernel_size=5),
            nn.Conv2d(3, 20, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(20, 50, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
             #hz added
            nn.Conv2d(50, 100, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(100, 200, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
        )

        self.feature_extractor_part2 = nn.Sequential(
#             nn.Linear(50 * 4 * 4, self.L),
            nn.Linear(200 * 4 * 4, self.L),
            nn.ReLU(),
        )

        self.feature_extractor_part3 = nn.Sequential(

            nn.Linear(self.L, self.B),
            nn.ReLU(),
            nn.Linear(self.B, self.L),
            nn.ReLU(),
        )

        self.attention_V = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.attention_U = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

#         self.attention_weights = nn.Linear(self.D, self.K)

        self.classifier = nn.Sequential(
#             nn.Linear(self.L*self.K, 1),
            nn.Linear(self.L*self.K, self.num_classes),
#             nn.Sigmoid()
        )

    def forward(self, x):

#         print('Inside forward: input x shape: {}'.format(x.shape))
        x = x.squeeze(0)
#         print('Inside forward: after squeeze x shape: {}'.format(x.shape))

        H = self.feature_extractor_part1(x)
#         print('Inside forward: after feature_extractor_part1 H shape: {}'.format(H.shape))


#         H = H.view(-1, 50 * 4 * 4)
        H = H.view(-1, 200 * 4 * 4)
#         print('Inside forward: after view H shape: {}'.format(H.shape))

        H = self.feature_extractor_part2(H)  # NxL
#         print('Inside forward: after feature_extractor_part2 H shape: {}'.format(H.shape))

        A_V = self.attention_V(H)  # NxK
#         print('Inside forward: A_V is {}, shape: {}'.format(A_V, A_V.shape))

        A_V = torch.transpose(A_V, 1, 0)  # KxN
#         print('Inside forward: A_V is {}, shape: {}'.format(A_V, A_V.shape))

        A_V = F.softmax(A_V, dim=1)  # softmax over N
#         print('Inside forward: A_V (View) is {}, shape: {}'.format(A_V, A_V.shape))


        H = self.feature_extractor_part3(H)

        A_U = self.attention_U(H)  # NxK
#         print('Inside forward: A_U is {}, shape: {}'.format(A_U, A_U.shape))

        A_U = torch.transpose(A_U, 1, 0)  # KxN
#         print('Inside forward: A_U is {}, shape: {}'.format(A_U, A_U.shape))

        A_U = F.softmax(A_U, dim=1)  # softmax over N
#         print('Inside forward: A_U (Diagnosis) is {}, shape: {}'.format(A_U, A_U.shape))

#         A = A_V * A_U
#         print('Inside forward: final A is {}, shape: {}'.format(A, A.shape))
        A = torch.exp(torch.log(A_V) + torch.log(A_U)) #numerically more stable?

        A = A/torch.sum(A)
#         A = F.softmax(A, dim=1)
#         print('Inside forward: final A is {}, shape: {}'.format(A, A.shape))
#         A = self.attention_weights(A_V * A_U) # element wise multiplication # NxK
#         print('Inside forward: A is {}, shape: {}'.format(A, A.shape))

#         A = torch.transpose(A, 1, 0)  # KxN
# #         print('Inside forward: A is {}, shape: {}'.format(A, A.shape))

#         A = F.softmax(A, dim=1)  # softmax over N
# #         print('Inside forward: A is {}, shape: {}'.format(A, A.shape))

        M = torch.mm(A, H)  # KxL #M can be regarded as final representation of this bag
#         print('Inside forward: M is {}, shape: {}'.format(M, M.shape))

        out = self.classifier(M)


        return out, A_V #only view regularize one branch of the attention weights



#### SAMIL Helpers
The following methods are from the paper's Github repo. Specifically the `src/SAMIL/main.py` file.

In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torchvision import transforms

from torch.utils.tensorboard import SummaryWriter

logger = logging.getLogger(__name__)

def str2bool(s):
    if s == 'True':
        return True
    elif s == 'False':
        return False
    else:
        raise NameError('Bad string')

def save_checkpoint(state, checkpoint_dir, filename='last_checkpoint.pth.tar'):
    '''last_checkpoint.pth.tar or xxx_model_best.pth.tar'''

    filepath = os.path.join(checkpoint_dir, filename)
    torch.save(state, filepath)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def get_cosine_schedule_with_warmup(optimizer,
                                    lr_warmup_epochs,
                                    lr_cycle_epochs, #total train epochs
                                    num_cycles=7./16.,
                                    last_epoch=-1):
    def _lr_lambda(current_epoch):
        if current_epoch < lr_warmup_epochs:
            return float(current_epoch) / float(max(1, lr_warmup_epochs))
#         no_progress = float(current_epoch - lr_warmup_epochs) / \
#             float(max(1, float(lr_cycle_epochs) - lr_warmup_epochs))

        #see if using restart
        ###############################################################
        if current_epoch%lr_cycle_epochs==0:
            current_cycle_epoch=lr_cycle_epochs
        else:
            current_cycle_epoch = current_epoch%lr_cycle_epochs

        no_progress = float(current_cycle_epoch - lr_warmup_epochs) / \
            float(max(1, float(lr_cycle_epochs) - lr_warmup_epochs))
        #################################################################

        return max(0., math.cos(math.pi * num_cycles * no_progress))

    return LambdaLR(optimizer, _lr_lambda, last_epoch)

def get_fixed_lr(optimizer,
                lr_warmup_epochs,
                lr_cycle_epochs, #total train iterations
                num_cycles=7./16.,
                last_epoch=-1):
    def _lr_lambda(current_epoch):

        return 1.0

    return LambdaLR(optimizer, _lr_lambda, last_epoch)

def create_view_model(args):

    view_model = build_wideresnet(depth=28,
                                        widen_factor=2,
                                        dropout=0.0,
                                        num_classes=3)

    logger.info("Total params for View Model: {:.2f}M".format(
        sum(p.numel() for p in view_model.parameters())/1e6))


    #load the saved checkpoint
    if args['data_seed']==0:
        args['view_checkpoint_path'] = os.path.join(args['checkpoint_dir'], 'view_classifier', 'seed0_model_best.pth.tar')
    elif args['data_seed']==1:
        args['view_checkpoint_path'] = os.path.join(args['checkpoint_dir'], 'view_classifier', 'seed1_model_best.pth.tar')
    elif args['data_seed']==2:
        args['view_checkpoint_path'] = os.path.join(args['checkpoint_dir'], 'view_classifier', 'seed2_model_best.pth.tar')
    else:
        raise NameError('?')


    view_checkpoint = torch.load(args['view_checkpoint_path'], map_location=device)

    view_model.load_state_dict(view_checkpoint['ema_state_dict'])

    view_model.eval()

    return view_model

def create_model(args):
    model = SAMIL()

    if args['MIL_checkpoint_path'] !='':
        print('!!!!!!!!!!!!!!!!!!!!!initializing from pretrained checkpoint!!!!!!!!!!!!!!!!!!!!!')
        pretrained_dict = torch.load(args['MIL_checkpoint_path'], map_location=device)

        #https://discuss.pytorch.org/t/dataparallel-changes-parameter-names-issue-with-load-state-dict/60211
        #rename tensor in the pretrained dict
        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in pretrained_dict.items():
#                         print(k)
            if 'encoder_q' in k:
#                             print('!extract: {}'.format(k))
                name = '.'.join(k.split('.')[1:])
#                             print('new_name: {}'.format(name))
                new_state_dict[name] = v

        model_dict = model.state_dict()

        new_state_dict = {k: v for k, v in new_state_dict.items() if k in model_dict}
        model_dict.update(new_state_dict)

        # 3. load the new state dict
        model.load_state_dict(model_dict)



    logger.info("Total params: {:.2f}M".format(
        sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6))

    return model

## SAMIL Training

The method below sets up various arguments around pretraining. The paper explores three methods of training: No Pretraining, pre training the Feature Extrator (to learn instance-level representations), and pre training the study-level representations of all *K* images in a routine echocardiogram.

In [18]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO
)

def setup_samil_train(args):

    if args['training_seed'] is not None:
        print('setting training seed{}'.format(args['training_seed']), flush=True)
        set_seed(args['training_seed'])


    if args['Pretrained'] == 'Whole':

        if args['data_seed']==0:
            args['MIL_checkpoint_path'] = os.path.join(args['checkpoint_dir'],'MOCO_Pretraining_StudyLevel', 'seed0_checkpoint.pt')
        elif args['data_seed']==1:
            args['MIL_checkpoint_path'] = os.path.join(args['checkpoint_dir'],'MOCO_Pretraining_StudyLevel', 'seed1_checkpoint.pt')
        elif args['data_seed']==2:
            args['MIL_checkpoint_path'] = os.path.join(args['checkpoint_dir'],'MOCO_Pretraining_StudyLevel', 'seed2_checkpoint.pt')
        else:
            raise NameError('NOT VALID PRETRAINED MODEL')

    elif args['Pretrained'] == 'FeatureExtractor1':

        if args['data_seed']==0:
            args['MIL_checkpoint_path']=os.path.join(args['checkpoint_dir'], 'MOCO_Pretraining_ImageLevel', 'seed0_checkpoint.pt')
        elif args['data_seed']==1:
            args['MIL_checkpoint_path']=os.path.join(args['checkpoint_dir'], 'MOCO_Pretraining_ImageLevel', 'seed1_checkpoint.pt')
        elif args['data_seed']==2:
            args['MIL_checkpoint_path']=os.path.join(args['checkpoint_dir'], 'MOCO_Pretraining_ImageLevel', 'seed2_checkpoint.pt')
        else:
            raise NameError('NOT VALID PRETRAINED MODEL')


    elif args['Pretrained'] == 'NoPretrain':
        args['MIL_checkpoint_path']=''

    else:
        raise NameError('invalid pretrain option')

    if args['use_class_weights'] == 'True':
        print('!!!!!!!!Using pre-calculated class weights!!!!!!!!')

        #indeed, every split should have the same class weight for diagnosis by our dataset construction
        if args['data_seed'] == 0 and args['development_size'] == 'DEV479':
            args['class_weights'] = '0.463,0.342,0.195'
        elif args['data_seed'] == 1 and args['development_size'] == 'DEV479':
            args['class_weights'] = '0.463,0.342,0.195'
        elif args['data_seed'] == 2 and args['development_size'] == 'DEV479':
            args['class_weights'] = '0.463,0.342,0.195'
        else:
            raise NameError('not valid class weights setting')

    else:
        args['class_weights'] = '1.0,1.0,1.0'
        print('?????????Not using pre-calculated class weights?????????')

    experiment_name = "{}".format(args['Pretrained'])

    args['experiment_dir'] = os.path.join(args['train_dir'], experiment_name)

    if args['resume'] != 'None':
        args['resume_checkpoint_fullpath'] = os.path.join(args['experiment_dir'], args['resume'])
        print('args.resume_checkpoint_fullpath: {}'.format(args['resume_checkpoint_fullpath']))
    else:
        args['resume_checkpoint_fullpath'] = None

    os.makedirs(args['experiment_dir'], exist_ok=True)
    args['writer'] = SummaryWriter(args['experiment_dir'])

    brief_summary = {}
    brief_summary['val_progression_view'] = {}

    brief_summary['dataset_name'] = args['dataset_name']
    brief_summary['algorithm'] = 'Echo_MIL'
    brief_summary['hyperparameters'] = {
        'train_epoch': args['train_epoch'],
        'optimizer': args['optimizer_type'],
        'lr': args['lr'],
        'wd': args['wd'],
        'T':args['T'],
        'lambda_ViewRegularization':args['lambda_ViewRegularization']
    }

    return args, brief_summary

#### Train One Epoch and Early Stop Logic

The code block below contains the methods to train a single epoch and the early stop logic.

In [19]:
from copy import deepcopy
from sklearn.metrics import balanced_accuracy_score

import torch

class ModelEMA(object):
    def __init__(self, args, model, decay):
        self.ema = deepcopy(model)
        self.ema.to(args['device'])
        self.ema.eval()
        self.decay = decay
        self.ema_has_module = hasattr(self.ema, 'module')
        # Fix EMA. https://github.com/valencebond/FixMatch_pytorch thank you!
        self.param_keys = [k for k, _ in self.ema.named_parameters()]
        self.buffer_keys = [k for k, _ in self.ema.named_buffers()]

        print('self.param_keys: {}'.format(self.param_keys))
        print('self.buffer_keys: {}'.format(self.buffer_keys))

        for p in self.ema.parameters():
#             print('Inside ModelEMA, p dtype is {}'.format(p.dtype))
            p.requires_grad_(False)


    def update(self, model):
        needs_module = hasattr(model, 'module') and not self.ema_has_module
        with torch.no_grad():
            msd = model.state_dict()
            esd = self.ema.state_dict()
            for k in self.param_keys:
                if needs_module:
                    j = 'module.' + k
                else:
                    j = k
                model_v = msd[j].detach()
                ema_v = esd[k]
                esd[k].copy_(ema_v * self.decay + (1. - self.decay) * model_v)

            for k in self.buffer_keys:
                if needs_module:
                    j = 'module.' + k
                else:
                    j = k
                esd[k].copy_(msd[j])

import time
from tqdm import tqdm
import torch.nn.functional as F

import logging
from sklearn.metrics import confusion_matrix as sklearn_cm
import numpy as np
import os
import pickle

import torch
import torch.nn as nn

import numpy as np

from sklearn.metrics import confusion_matrix as sklearn_cm


class EarlyStopping:
    """Early stops the training if validation acc doesn't improve after a given patience."""

    def __init__(self, patience=300, initial_count=0, delta=0):

        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 20
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0

        """

        self.patience = patience
        self.counter = initial_count
        self.best_score = None
        self.early_stop = False
        self.delta = delta


    def __call__(self, val_acc):

        score = val_acc

        if self.best_score is None:
            self.best_score = score

        elif score <= self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

        else:
            self.best_score = score
            self.counter = 0

        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!counter: {}, score: {}, best_score: {}'.format(self.counter, score, self.best_score))

        return self.counter


def train_one_epoch(args, weights, train_loader, model, ema_model, view_model, optimizer, scheduler, epoch):

    args['writer'].add_scalar('train/lr', scheduler.get_last_lr()[0], epoch)

    model.train()

    TotalLoss_this_epoch, LabeledCELoss_this_epoch, ViewRegularizationLoss_this_epoch, scaled_ViewRegularizationLoss_this_epoch = [], [], [], []

    train_iter = iter(train_loader)
    n_steps_per_epoch = 360 #360 train studies, batch size 1
    p_bar = tqdm(range(n_steps_per_epoch), disable=False)

#     for batch_idx, (data, bag_label, view_relevance) in enumerate(tqdm(train_loader)):
    for batch_idx in range(n_steps_per_epoch):

        try:
            data, bag_label = next(train_iter)
        except:
            train_iter = iter(train_loader)
            data, bag_label = next(train_iter)

#         print('batch_idx: {}'.format(batch_idx))

#         print('type(data): {}, data.size: {}, require grad: {}'.format(type(data), data.size(), data.requires_grad))
#         print('type(bag_label): {}, bag_label: {}'.format(type(bag_label), bag_label))
#         print('type(view_relevance): {}, view_relevance: {}'.format(type(view_relevance), view_relevance))
        data, bag_label = data.to(args['device']), bag_label.to(args['device'])


        outputs, attentions = model(data)

        log_attentions = torch.log(attentions)


        with torch.no_grad():
            view_predictions = view_model(data.squeeze(0))
            softmax_view_predictions = F.softmax(view_predictions, dim=1)
            predicted_relevance = softmax_view_predictions[:, :2]
            predicted_relevance = torch.sum(predicted_relevance, dim=1)
            predicted_relative_relevance = F.softmax(predicted_relevance/args['T'])
            predicted_relative_relevance = predicted_relative_relevance.unsqueeze(0)


        #element shape in F.cross_entropy: prediction torch.size([batch_size, num_classes]) and true label torch.size([batch_size])
        if args['use_class_weights'] == 'True':
            LabeledCELoss = F.cross_entropy(outputs, bag_label, weights, reduction='mean')
        else:
            LabeledCELoss = F.cross_entropy(outputs, bag_label, reduction='mean')


#         parser.add_argument('--ViewRegularization_warmup_pos', default=0.4, type=float, help='position at which view regularization loss warmup ends') #following MixMatch and FixMatch repo

# parser.add_argument('--ViewRegularization_warmup_schedule_type', default='NoWarmup', choices=['NoWarmup', 'Linear', 'Sigmoid', ], type=str)

        #ViewRegularization warmup schedule choice
        if args['ViewRegularization_warmup_schedule_type'] == 'NoWarmup':
            current_warmup = 1
        elif args['ViewRegularization_warmup_schedule_type'] == 'Linear':
            current_warmup = np.clip(epoch/(float(args['ViewRegularization_warmup_pos']) * args['train_epoch']), 0, 1)
        elif args['ViewRegularization_warmup_schedule_type'] == 'Sigmoid':
            current_warmup = math.exp(-5 * (1 - min(epoch/(float(args['ViewRegularization_warmup_pos']) * args['train_epoch']), 1))**2)
        else:
            raise NameError('Not supported ViewRegularization warmup schedule')



        ViewRegularizationLoss = F.kl_div(input=log_attentions, target=predicted_relative_relevance, log_target=False, reduction='batchmean')

        # backward pass
        total_loss = LabeledCELoss + args['lambda_ViewRegularization'] * ViewRegularizationLoss * current_warmup

        total_loss.backward()



        TotalLoss_this_epoch.append(total_loss.item())
        LabeledCELoss_this_epoch.append(LabeledCELoss.item())
        ViewRegularizationLoss_this_epoch.append(ViewRegularizationLoss.item())
        scaled_ViewRegularizationLoss_this_epoch.append(args['lambda_ViewRegularization'] * ViewRegularizationLoss.item() * current_warmup)

        # step
        optimizer.step()

        #update ema model
        ema_model.update(model)

        model.zero_grad()

    scheduler.step()

    return TotalLoss_this_epoch, LabeledCELoss_this_epoch, ViewRegularizationLoss_this_epoch, scaled_ViewRegularizationLoss_this_epoch



#regular eval_model
def eval_model(args, data_loader, raw_model, ema_model, epoch):

    raw_model.eval()
    ema_model.eval()

    data_loader = tqdm(data_loader, disable=False)

    with torch.no_grad():
        total_targets = []
        total_raw_outputs = []
        total_ema_outputs = []


        for batch_idx, (data, bag_label) in enumerate(data_loader):

#             print('EVAL type(data): {}, data.size: {}, require grad: {}'.format(type(data), data.size(), data.requires_grad))
#             print('EVAL type(bag_label): {}, bag_label: {}'.format(type(bag_label), bag_label))

            data, bag_label = data.to(args['device']), bag_label.to(args['device'])

            raw_outputs, raw_attention_weights = raw_model(data)
            ema_outputs, ema_attention_weights = ema_model(data)
#             print('target is {}, raw_outputs is: {}, ema_outputs is {}'.format(bag_label, raw_outputs, ema_outputs))

            total_targets.append(bag_label.detach().cpu())
            total_raw_outputs.append(raw_outputs.detach().cpu())
            total_ema_outputs.append(ema_outputs.detach().cpu())


        total_targets = np.concatenate(total_targets, axis=0)
        total_raw_outputs = np.concatenate(total_raw_outputs, axis=0)
        total_ema_outputs = np.concatenate(total_ema_outputs, axis=0)
#         print('RegularEval total_targets: {}'.format(total_targets))
#         print('RegularEval total_raw_outputs: {}'.format(total_raw_outputs))
#         print('RegularEval total_ema_outputs: {}'.format(total_ema_outputs))

        raw_Bacc = calculate_balanced_accuracy(total_raw_outputs, total_targets)
        ema_Bacc = calculate_balanced_accuracy(total_ema_outputs, total_targets)

#         print('raw Bacc this evaluation step: {}'.format(raw_Bacc), flush=True)
#         print('ema Bacc this evaluation step: {}'.format(ema_Bacc), flush=True)


    return raw_Bacc, ema_Bacc, total_targets, total_raw_outputs, total_ema_outputs

def eval_model_test(args, data_loader, raw_model):
    raw_model.eval()

    with torch.no_grad():
        ground_truth_labels = []
        pred_labels = []

        for data, bag_label in data_loader:
            data, bag_label = data.to(device), bag_label.to(device)

            pred_logit, _ = raw_model(data)

            pred_label = torch.softmax(pred_logit, dim=-1)
            pred_label = torch.argmax(pred_label).item()

            pred_labels.append(pred_label)
            ground_truth_labels.append(bag_label.item())

        bal_acc = balanced_accuracy_score(ground_truth_labels, pred_labels)

    return bal_acc

def calculate_balanced_accuracy(prediction, true_target, return_type = 'only balanced_accuracy'):

    confusion_matrix = sklearn_cm(true_target, prediction.argmax(1))
    n_class = confusion_matrix.shape[0]
    print('Inside calculate_balanced_accuracy, {} classes passed in'.format(n_class), flush=True)

    assert n_class==3

    recalls = []
    for i in range(n_class):
        recall = confusion_matrix[i,i]/np.sum(confusion_matrix[i])
        recalls.append(recall)
        print('class{} recall: {}'.format(i, recall), flush=True)

    balanced_accuracy = np.mean(np.array(recalls))


    if return_type == 'all':
#         return balanced_accuracy * 100, class0_recall * 100, class1_recall * 100, class2_recall * 100
        return balanced_accuracy * 100, recalls

    elif return_type == 'only balanced_accuracy':
        return balanced_accuracy * 100
    else:
        raise NameError('Unsupported return_type in this calculate_balanced_accuracy fn')


 #shared helper fct across different algos
def save_pickle(save_dir, save_file_name, data):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    data_save_fullpath = os.path.join(save_dir, save_file_name)
    with open(data_save_fullpath, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)


#### Training Runner
The code block below contains the logic to train the model, one epoch at a time, with early stop, and the logic to write the results out. This method is called in subsequent training blocks after the training arguments are defined.

In [20]:
import json

def train_samil(args):
    best_val_ema_Bacc = 0
    best_test_ema_Bacc_at_val = 0
    best_train_ema_Bacc_at_val = 0

    best_val_raw_Bacc = 0
    best_test_raw_Bacc_at_val = 0
    best_train_raw_Bacc_at_val = 0


    current_count=0

    if os.path.isfile(args.get('resume_checkpoint_fullpath')):

        print('Resuming from checkpoint: {}'.format(args.get('resume_checkpoint_fullpath')))

        checkpoint = torch.load(args['resume_checkpoint_fullpath'])
        args['start_epoch'] = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        ema_model.ema.load_state_dict(checkpoint['ema_state_dict'])
        current_count = checkpoint['current_count']
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])

        best_val_ema_Bacc = checkpoint['val_progression_view']['best_val_ema_Bacc']
        best_test_ema_Bacc_at_val = checkpoint['val_progression_view']['best_test_ema_Bacc_at_val']
        best_train_ema_Bacc_at_val = checkpoint['val_progression_view']['best_train_ema_Bacc_at_val']

        best_val_raw_Bacc = checkpoint['val_progression_view']['best_val_raw_Bacc']
        best_test_raw_Bacc_at_val = checkpoint['val_progression_view']['best_test_raw_Bacc_at_val']
        best_train_raw_Bacc_at_val = checkpoint['val_progression_view']['best_train_raw_Bacc_at_val']


    else:
        print('!!!!Does not have checkpoint yet!!!!')


    logger.info("***** Running training *****")
    logger.info(f"  Task = {args['dataset_name']}")
    logger.info(f"  Num Epochs = {args['train_epoch']}")
    logger.info(f"  Total optimization steps = {args['train_epoch'] * len(train_dataset)}")

    train_loss_dict = dict()
    train_loss_dict['Totalloss'] = []
    train_loss_dict['LabeledCEloss'] = []
    train_loss_dict['ViewRegularizationLoss'] = []

    early_stopping = EarlyStopping(patience=args['patience'], initial_count=current_count)
    early_stopping_warmup = args['early_stopping_warmup']

    for epoch in tqdm(range(args['start_epoch'], args['train_epoch'])):
        val_predictions_save_dict = dict()
        test_predictions_save_dict = dict()
        train_predictions_save_dict = dict()

        TotalLoss_list, LabeledCEloss_list, ViewRegularizationLoss_list, scaled_ViewRegularizationLoss_list = train_one_epoch(args, weights, train_loader, model, ema_model, view_model, optimizer, scheduler, epoch)
        train_loss_dict['Totalloss'].extend(TotalLoss_list)
        train_loss_dict['LabeledCEloss'].extend(LabeledCEloss_list)
        train_loss_dict['ViewRegularizationLoss'].extend(ViewRegularizationLoss_list)

        if epoch % args['eval_every_Xepoch'] == 0:
            val_raw_Bacc, val_ema_Bacc, val_true_labels, val_raw_predictions, val_ema_predictions = eval_model(args, val_loader, model, ema_model.ema, epoch)
            val_predictions_save_dict['raw_Bacc'] = val_raw_Bacc
            val_predictions_save_dict['ema_Bacc'] = val_ema_Bacc
            val_predictions_save_dict['true_labels'] = val_true_labels
            val_predictions_save_dict['raw_predictions'] = val_raw_predictions
            val_predictions_save_dict['ema_predictions'] = val_ema_predictions

            test_raw_Bacc, test_ema_Bacc, test_true_labels, test_raw_predictions, test_ema_predictions = eval_model(args, test_loader, model, ema_model.ema, epoch)

            test_predictions_save_dict['raw_Bacc'] = test_raw_Bacc
            test_predictions_save_dict['ema_Bacc'] = test_ema_Bacc
            test_predictions_save_dict['true_labels'] = test_true_labels
            test_predictions_save_dict['raw_predictions'] = test_raw_predictions
            test_predictions_save_dict['ema_predictions'] = test_ema_predictions

            train_raw_Bacc, train_ema_Bacc, train_true_labels, train_raw_predictions, train_ema_predictions = eval_model(args, trainmemory_loader, model, ema_model.ema, epoch)

            train_predictions_save_dict['raw_Bacc'] = train_raw_Bacc
            train_predictions_save_dict['ema_Bacc'] = train_ema_Bacc
            train_predictions_save_dict['true_labels'] = train_true_labels
            train_predictions_save_dict['raw_predictions'] = train_raw_predictions
            train_predictions_save_dict['ema_predictions'] = train_ema_predictions

            if val_raw_Bacc > best_val_raw_Bacc:

                best_val_raw_Bacc = val_raw_Bacc
                best_test_raw_Bacc_at_val = test_raw_Bacc
                best_train_raw_Bacc_at_val = train_raw_Bacc

                save_pickle(os.path.join(args['experiment_dir'], 'val_progression_view', 'best_predictions_at_raw_val'), 'val_predictions.pkl', val_predictions_save_dict)

                save_pickle(os.path.join(args['experiment_dir'], 'val_progression_view', 'best_predictions_at_raw_val'), 'test_predictions.pkl', test_predictions_save_dict)


                save_pickle(os.path.join(args['experiment_dir'], 'val_progression_view', 'best_predictions_at_raw_val'), 'train_predictions.pkl', train_predictions_save_dict)

                save_checkpoint(
                {
                'epoch': epoch+1,
                'state_dict': model.state_dict(),
                'ema_state_dict': ema_model.ema.state_dict(),
                'current_count':current_count,
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict(),

                'val_progression_view':
                    {'epoch': epoch+1,
                    'best_val_ema_Bacc': best_val_ema_Bacc,
                    'best_val_raw_Bacc': best_val_raw_Bacc,
                    'best_test_ema_Bacc_at_val': best_test_ema_Bacc_at_val,
                    'best_test_raw_Bacc_at_val': best_test_raw_Bacc_at_val,
                    'best_train_ema_Bacc_at_val': best_train_ema_Bacc_at_val,
                    'best_train_raw_Bacc_at_val': best_train_raw_Bacc_at_val,
                      },

                }, args['experiment_dir'], filename='val_progression_view/best_predictions_at_raw_val/best_model.pth.tar')


            if val_ema_Bacc > best_val_ema_Bacc:

                best_val_ema_Bacc = val_ema_Bacc
                best_test_ema_Bacc_at_val = test_ema_Bacc
                best_train_ema_Bacc_at_val = train_ema_Bacc

                save_pickle(os.path.join(args['experiment_dir'], 'val_progression_view', 'best_predictions_at_ema_val'), 'val_predictions.pkl', val_predictions_save_dict)

                save_pickle(os.path.join(args['experiment_dir'], 'val_progression_view', 'best_predictions_at_ema_val'), 'test_predictions.pkl', test_predictions_save_dict)

                save_pickle(os.path.join(args['experiment_dir'], 'val_progression_view', 'best_predictions_at_ema_val'), 'train_predictions.pkl', train_predictions_save_dict)

                save_checkpoint(
                {
                'epoch': epoch+1,
                'state_dict': model.state_dict(),
                'ema_state_dict': ema_model.ema.state_dict(),
                'current_count':current_count,
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict(),

                'val_progression_view':
                    {'epoch': epoch+1,
                    #regular val
                    'best_val_ema_Bacc': best_val_ema_Bacc,
                    'best_val_raw_Bacc': best_val_raw_Bacc,
                    'best_test_ema_Bacc_at_val': best_test_ema_Bacc_at_val,
                    'best_test_raw_Bacc_at_val': best_test_raw_Bacc_at_val,
                    'best_train_ema_Bacc_at_val': best_train_ema_Bacc_at_val,
                    'best_train_raw_Bacc_at_val': best_train_raw_Bacc_at_val,
                      },

                }, args['experiment_dir'], filename='val_progression_view/best_predictions_at_ema_val/best_model.pth.tar')



            logger.info('val progression view:')
            logger.info('At RAW Best val, validation/test/train %.2f %.2f %.2f' % (best_val_raw_Bacc, best_test_raw_Bacc_at_val, best_train_raw_Bacc_at_val))
            logger.info('At EMA Best val, validation/test/train %.2f %.2f %.2f' % (best_val_ema_Bacc, best_test_ema_Bacc_at_val, best_train_ema_Bacc_at_val))

            args['writer'].add_scalar('train/1.train_raw_Bacc', train_raw_Bacc, epoch)
            args['writer'].add_scalar('train/1.train_ema_Bacc', train_ema_Bacc, epoch)
            args['writer'].add_scalar('train/1.LabeledCEloss', np.mean(LabeledCEloss_list), epoch)

            args['writer'].add_scalar('val/1.val_raw_Bacc', val_raw_Bacc, epoch)
            args['writer'].add_scalar('val/2.val_ema_Bacc', val_ema_Bacc, epoch)


            args['writer'].add_scalar('test/1.test_raw_Bacc', test_raw_Bacc, epoch)
            args['writer'].add_scalar('test/2.test_ema_Bacc', test_ema_Bacc, epoch)

            brief_summary['val_progression_view']['best_val_ema_Bacc'] = best_val_ema_Bacc
            brief_summary['val_progression_view']['best_val_raw_Bacc'] = best_val_raw_Bacc
            brief_summary['val_progression_view']['best_test_ema_Bacc_at_val'] = best_test_ema_Bacc_at_val
            brief_summary['val_progression_view']['best_test_raw_Bacc_at_val'] = best_test_raw_Bacc_at_val
            brief_summary['val_progression_view']['best_train_ema_Bacc_at_val'] = best_train_ema_Bacc_at_val
            brief_summary['val_progression_view']['best_train_raw_Bacc_at_val'] = best_train_raw_Bacc_at_val


            with open(os.path.join(args['experiment_dir'], "brief_summary.json"), "w") as f:
                json.dump(brief_summary, f)

            if epoch > early_stopping_warmup:
                current_count = early_stopping(val_ema_Bacc)

            save_checkpoint(
                {
                'epoch': epoch+1,
                'state_dict': model.state_dict(),
                'ema_state_dict': ema_model.ema.state_dict(),
                'current_count':current_count,
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict(),

                'val_progression_view':
                    {'epoch': epoch+1,
                    #regular val
                    'best_val_ema_Bacc': best_val_ema_Bacc,
                    'best_val_raw_Bacc': best_val_raw_Bacc,
                    'best_test_ema_Bacc_at_val': best_test_ema_Bacc_at_val,
                    'best_test_raw_Bacc_at_val': best_test_raw_Bacc_at_val,
                    'best_train_ema_Bacc_at_val': best_train_ema_Bacc_at_val,
                    'best_train_raw_Bacc_at_val': best_train_raw_Bacc_at_val,
                      },


                }, args['experiment_dir'], filename='last_checkpoint.pth.tar')


            if early_stopping.early_stop:
                break

    brief_summary['val_progression_view']['best_val_ema_Bacc'] = best_val_ema_Bacc
    brief_summary['val_progression_view']['best_val_raw_Bacc'] = best_val_raw_Bacc
    brief_summary['val_progression_view']['best_test_ema_Bacc_at_val'] = best_test_ema_Bacc_at_val
    brief_summary['val_progression_view']['best_test_raw_Bacc_at_val'] = best_test_raw_Bacc_at_val
    brief_summary['val_progression_view']['best_train_ema_Bacc_at_val'] = best_train_ema_Bacc_at_val
    brief_summary['val_progression_view']['best_train_raw_Bacc_at_val'] = best_train_raw_Bacc_at_val



    args['writer'].close()

    with open(os.path.join(args['experiment_dir'], "brief_summary.json"), "w") as f:
        json.dump(brief_summary, f)

### Configure SAMIL Arguments for Training

The code block below configures the arguments for training the SAMIL model per the specified hyperparameters in the paper for Split 1 and the Github repo here: [Hyperparameters](https://github.com/tufts-ml/SAMIL/blob/main/Hyperparameters/Hyperparameters.txt)

| SAMIL (with study-level SSL) | split1 | split2 | split3 |
| ---------------------------- | ------ | ------ | ------ |
| Learning rate                | 0.0008 | 0.0005 | 0.0005 |
| Weight decay                 | 0.0001 | 0.0001 | 0.001  |
| Temperature T                | 0.1    | 0.05   | 0.1    |
| λ<sub>sA</sub>               | 15.0   | 20.0   | 20.0   |
| Learning rate schedule       | cosine | cosine | cosine |

Table C.1: Hyperparameter settings for SAMIL across different data splits.

#### Training Type 1: No Pretraining

**Please change hyperparameters and epochs as needed.**

The epoch count defined in the paper is 2,000. To test the training below, set `train_epoch` to 1.



In [23]:
RUNS_DIR = '/content/runs/'

args = {
    'training_seed': 0,
    'Pretrained': 'NoPretrain',
    'data_seed': 0,
    'checkpoint_dir': MODEL_CHECKPOINTS,
    'MIL_checkpoint_path': '',
    'use_class_weights': 'True',
    'ViewRegularization_warmup_schedule_type': 'Linear',
    'optimizer_type': 'SGD',
    'lr_schedule_type': 'CosineLR',
    'lr_cycle_epochs': 1,
    'lr': 0.0008, # learning rate
    'wd': 0.0001, # weight decay
    'T': 0.1, # tempertature
    'lambda_ViewRegularization': 15.0, # λsA
    'train_dir': RUNS_DIR + 'SAMIL',
    'resume': 'last_checkpoint.pth.tar',
    'dataset_name': 'echo',
    'train_epoch': 2000, # number of epochs, 2000 defined in the paper. CHANGE ME!
    'development_size': 'DEV479',
    'lr_warmup_epochs': 0,
    'ema_decay': 0.999,
    'device': device,
    'start_epoch': 0,
    'patience': 200,
    'early_stopping_warmup': 200,
    'ViewRegularization_warmup_pos': 0.4,
    'eval_every_Xepoch': 1
}

args, brief_summary = setup_samil_train(args)

weights = args['class_weights']
weights = [float(i) for i in weights.split(',')]
weights = torch.Tensor(weights)
weights = weights.to(device)

#load the view model, the output is unnormalized logits, need to use softmax on the output
view_model = create_view_model(args)
view_model.to(device)

model = create_model(args)
model.to(device)

no_decay = ['bias', 'bn']
grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay)], 'weight_decay': args['wd']},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

if args['optimizer_type'] == 'SGD':
    optimizer = optim.SGD(grouped_parameters, lr=args['lr'],
                          momentum=0.9, nesterov=True)

elif args['optimizer_type'] == 'Adam':
    optimizer = optim.Adam(grouped_parameters, lr=args['lr'])

elif args['optimizer_type'] == 'AdamW':
    optimizer = optim.AdamW(grouped_parameters, lr=args['lr'])

else:
    raise NameError('Not supported optimizer setting')

#lr_schedule_type choice
if args['lr_schedule_type'] == 'CosineLR':
    scheduler = get_cosine_schedule_with_warmup(optimizer, args['lr_warmup_epochs'], args['lr_cycle_epochs'])

elif args['lr_schedule_type'] == 'FixedLR':
    scheduler = get_fixed_lr(optimizer, args['lr_warmup_epochs'], args['lr_cycle_epochs'])

else:
    raise NameError('Not supported lr scheduler setting')


#instantiate the ema_model object
ema_model = ModelEMA(args, model, args['ema_decay'])


# !!! Start training
train_samil(args)

setting training seed0


INFO:__main__:Model: WideResNet 28x2


!!!!!!!!Using pre-calculated class weights!!!!!!!!
args.resume_checkpoint_fullpath: /content/runs/SAMIL/NoPretrain/last_checkpoint.pth.tar


INFO:__main__:Total params for View Model: 5.93M
INFO:__main__:Total params: 2.31M
INFO:__main__:***** Running training *****
INFO:__main__:  Task = echo
INFO:__main__:  Num Epochs = 2000
INFO:__main__:  Total optimization steps = 720000


self.param_keys: ['feature_extractor_part1.0.weight', 'feature_extractor_part1.0.bias', 'feature_extractor_part1.3.weight', 'feature_extractor_part1.3.bias', 'feature_extractor_part1.6.weight', 'feature_extractor_part1.6.bias', 'feature_extractor_part1.9.weight', 'feature_extractor_part1.9.bias', 'feature_extractor_part2.0.weight', 'feature_extractor_part2.0.bias', 'feature_extractor_part3.0.weight', 'feature_extractor_part3.0.bias', 'feature_extractor_part3.2.weight', 'feature_extractor_part3.2.bias', 'attention_V.0.weight', 'attention_V.0.bias', 'attention_V.2.weight', 'attention_V.2.bias', 'attention_U.0.weight', 'attention_U.0.bias', 'attention_U.2.weight', 'attention_U.2.bias', 'classifier.0.weight', 'classifier.0.bias']
self.buffer_keys: []
!!!!Does not have checkpoint yet!!!!


...

Inside calculate_balanced_accuracy, 3 classes passed in
class0 recall: 0.0
class1 recall: 0.0
class2 recall: 1.0
Inside calculate_balanced_accuracy, 3 classes passed in
class0 recall: 0.0
class1 recall: 0.0
class2 recall: 1.0





  0%|          | 0/360 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 1/360 [00:01<08:42,  1.46s/it]

  1%|          | 2/360 [00:01<04:01,  1.48it/s]

  1%|          | 3/360 [00:01<02:39,  2.23it/s]

  1%|          | 4/360 [00:01<01:52,  3.18it/s]

  2%|▏         | 6/360 [00:02<01:04,  5.46it/s]

  2%

Inside calculate_balanced_accuracy, 3 classes passed in
class0 recall: 0.0
class1 recall: 0.0
class2 recall: 1.0
Inside calculate_balanced_accuracy, 3 classes passed in
class0 recall: 0.0
class1 recall: 0.0
class2 recall: 1.0



INFO:__main__:val progression view:
INFO:__main__:At RAW Best val, validation/test/train 33.33 33.33 33.33
INFO:__main__:At EMA Best val, validation/test/train 33.33 33.33 33.33
  5%|▍         | 93/2000 [2:37:26<57:53:20, 109.28s/it]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/360 [00:00<?, ?it/s]<ipython-input-19-

KeyboardInterrupt: 

#### Type 2 - Image-Level Pre Training

In [ ]:
RUNS_DIR = '/content/runs/'

args = {
    'training_seed': 0,
    'Pretrained': 'Whole', # Options are Whole for Study Level, FeatureExtractor for Image Level, or NoPretail for no SSL.
    'data_seed': 0,
    'checkpoint_dir': MODEL_CHECKPOINTS,
    'MIL_checkpoint_path': '',
    'use_class_weights': 'True',
    'ViewRegularization_warmup_schedule_type': 'Linear',
    'optimizer_type': 'SGD',
    'lr_schedule_type': 'CosineLR',
    'lr_cycle_epochs': 1,
    'lr': 0.0008, # learning rate
    'wd': 0.0001, # weight decay
    'T': 0.1, # tempertature
    'lambda_ViewRegularization': 15.0, # λsA
    'train_dir': RUNS_DIR + 'SAMIL',
    'resume': 'last_checkpoint.pth.tar',
    'dataset_name': 'echo',
    'train_epoch': 2000, # number of epochs, 2000 defined in the paper. CHANGE ME!
    'development_size': 'DEV479',
    'lr_warmup_epochs': 0,
    'ema_decay': 0.999,
    'device': device,
    'start_epoch': 0,
    'patience': 200,
    'early_stopping_warmup': 200,
    'ViewRegularization_warmup_pos': 0.4,
    'eval_every_Xepoch': 1
}

args, brief_summary = setup_samil_train(args)

weights = args['class_weights']
weights = [float(i) for i in weights.split(',')]
weights = torch.Tensor(weights)
weights = weights.to(device)

#load the view model, the output is unnormalized logits, need to use softmax on the output
view_model = create_view_model(args)
view_model.to(device)

model = create_model(args)
model.to(device)

no_decay = ['bias', 'bn']
grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay)], 'weight_decay': args['wd']},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

if args['optimizer_type'] == 'SGD':
    optimizer = optim.SGD(grouped_parameters, lr=args['lr'],
                          momentum=0.9, nesterov=True)

elif args['optimizer_type'] == 'Adam':
    optimizer = optim.Adam(grouped_parameters, lr=args['lr'])

elif args['optimizer_type'] == 'AdamW':
    optimizer = optim.AdamW(grouped_parameters, lr=args['lr'])

else:
    raise NameError('Not supported optimizer setting')

#lr_schedule_type choice
if args['lr_schedule_type'] == 'CosineLR':
    scheduler = get_cosine_schedule_with_warmup(optimizer, args['lr_warmup_epochs'], args['lr_cycle_epochs'])

elif args['lr_schedule_type'] == 'FixedLR':
    scheduler = get_fixed_lr(optimizer, args['lr_warmup_epochs'], args['lr_cycle_epochs'])

else:
    raise NameError('Not supported lr scheduler setting')


#instantiate the ema_model object
ema_model = ModelEMA(args, model, args['ema_decay'])


# !!! Start training
train_samil(args)

#### Training Type 3 - Study-Level (i.e., Bag-Level) Pre Training

This training run trains the model with the two novel contributions of the paper. Namely, the self-supervised (SSL) pre-training of the entire study-level representations that builds upon MoCo (V2). MoCo is a "recent method for self-supervised image-level contrastive learning (img-CL). (Huang et., al, 2023).

In [1]:
RUNS_DIR = '/content/runs/'

args = {
    'training_seed': 0,
    'Pretrained': 'Whole', # Options are Whole for Study Level, FeatureExtractor for Image Level, or NoPretail for no SSL.
    'data_seed': 0,
    'checkpoint_dir': MODEL_CHECKPOINTS,
    'MIL_checkpoint_path': '',
    'use_class_weights': 'True',
    'ViewRegularization_warmup_schedule_type': 'Linear',
    'optimizer_type': 'SGD',
    'lr_schedule_type': 'CosineLR',
    'lr_cycle_epochs': 1,
    'lr': 0.0008, # learning rate
    'wd': 0.0001, # weight decay
    'T': 0.1, # tempertature
    'lambda_ViewRegularization': 15.0, # λsA
    'train_dir': RUNS_DIR + 'SAMIL',
    'resume': 'last_checkpoint.pth.tar',
    'dataset_name': 'echo',
    'train_epoch': 2000, # number of epochs, 2000 defined in the paper. CHANGE ME!
    'development_size': 'DEV479',
    'lr_warmup_epochs': 0,
    'ema_decay': 0.999,
    'device': device,
    'start_epoch': 0,
    'patience': 200,
    'early_stopping_warmup': 200,
    'ViewRegularization_warmup_pos': 0.4,
    'eval_every_Xepoch': 1
}

args, brief_summary = setup_samil_train(args)

weights = args['class_weights']
weights = [float(i) for i in weights.split(',')]
weights = torch.Tensor(weights)
weights = weights.to(device)

#load the view model, the output is unnormalized logits, need to use softmax on the output
view_model = create_view_model(args)
view_model.to(device)

model = create_model(args)
model.to(device)

no_decay = ['bias', 'bn']
grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay)], 'weight_decay': args['wd']},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

if args['optimizer_type'] == 'SGD':
    optimizer = optim.SGD(grouped_parameters, lr=args['lr'],
                          momentum=0.9, nesterov=True)

elif args['optimizer_type'] == 'Adam':
    optimizer = optim.Adam(grouped_parameters, lr=args['lr'])

elif args['optimizer_type'] == 'AdamW':
    optimizer = optim.AdamW(grouped_parameters, lr=args['lr'])

else:
    raise NameError('Not supported optimizer setting')

#lr_schedule_type choice
if args['lr_schedule_type'] == 'CosineLR':
    scheduler = get_cosine_schedule_with_warmup(optimizer, args['lr_warmup_epochs'], args['lr_cycle_epochs'])

elif args['lr_schedule_type'] == 'FixedLR':
    scheduler = get_fixed_lr(optimizer, args['lr_warmup_epochs'], args['lr_cycle_epochs'])

else:
    raise NameError('Not supported lr scheduler setting')


#instantiate the ema_model object
ema_model = ModelEMA(args, model, args['ema_decay'])


# !!! Start training
train_samil(args)

NameError: name 'MODEL_CHECKPOINTS' is not defined

# Results
In this section, you should finish training your model training or loading your trained model. That is a great experiment! You should share the results with others with necessary metrics and figures.

Please test and report results for all experiments that you run with:

*   specific numbers (accuracy, AUC, RMSE, etc)
*   figures (loss shrinkage, outputs from GAN, annotation or label of sample pictures, etc)


### Model comparison

#### SAMIL with No Pretraining

In [ ]:
target_balanced_accuracy = 0.727

load_model_path = os.path.join('runs/SAMIL/NoPretrain', "last_checkpoint.pth.tar")

loaded_SAMIL_Model = SAMIL().to(device)
loaded_SAMIL_Model.load_state_dict(torch.load(load_model_path)['ema_state_dict'])

loaded_SAMIL_Model.eval()

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=8)
bal_acc = eval_model_test(args, test_loader, loaded_SAMIL_Model)

print(f"Target bal_acc (from paper): {target_balanced_accuracy}")
print(f"Reproduced bal_acc (from our implementation): {bal_acc}")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Paper's bal_acc: 0.727
Bal_acc: 0.3333333333333333


#### SAMIL with Image Level Pretraining

In [ ]:
target_balanced_accuracy = 0.727

load_model_path = os.path.join('runs/SAMIL/FeatureExtractor1', "last_checkpoint.pth.tar")

loaded_SAMIL_Model = SAMIL().to(device)
loaded_SAMIL_Model.load_state_dict(torch.load(load_model_path)['ema_state_dict'])

loaded_SAMIL_Model.eval()

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=8)
bal_acc = eval_model_test(args, test_loader, loaded_SAMIL_Model)

print(f"Target bal_acc (from paper): {target_balanced_accuracy}")
print(f"Reproduced bal_acc (from our implementation): {bal_acc}")

#### SAMIL with Pretraining

In [ ]:
target_balanced_accuracy = 0.754

# Testing by loading a saved checkpoint
load_model_path = os.path.join('runs/SAMIL/Whole', "last_checkpoint.pth.tar")

loaded_SAMIL_Model = SAMIL().to(device)
loaded_SAMIL_Model.load_state_dict(torch.load(load_model_path)['ema_state_dict'])

loaded_SAMIL_Model.eval()

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=8)
bal_acc = eval_model_test(args, test_loader, loaded_SAMIL_Model)

print(f"Target bal_acc (from paper): {target_balanced_accuracy}")
print(f"Reproduced bal_acc (from our implementation): {bal_acc}")

# Discussion

In this section,you should discuss your work and make future plan. The discussion should address the following questions:
  * Make assessment that the paper is reproducible or not.
  * Explain why it is not reproducible if your results are kind negative.
  * Describe “What was easy” and “What was difficult” during the reproduction.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
  * What will you do in next phase.



# References

1.   Huang, Zhe, Benjamin S. Wessler, and Michael C. Hughes. "Detecting Heart Disease from Multi-View Ultrasound Images via Supervised Attention Multiple Instance Learning." Machine Learning for Healthcare Conference. PMLR, 2023. https://doi.org/10.48550/arXiv.2306.00003

